### Load Libraries: 

In [1]:
from models.inception_resnet_v1 import InceptionResnetV1

import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
from torch.autograd import Variable
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.nn as nn

import torchvision.utils as vutils
from io import open
import os
from PIL import Image
import sys

import torchvision.models as models
import pickle

import pandas as pd

import numpy as np

### Choose version of Facenet: 

In [19]:
############  CHOOSE HERE WHICH ALGORITHM YOU WANT: #################
                                                                    #
model = InceptionResnetV1(pretrained='vggface2')                    #
SaveAs = 'VggFace2'                                                 #
                    ##################                              #
                                                                    #
# model = InceptionResnetV1(pretrained='casia-webface')               #
# SaveAs = 'CasiaWebFace'                                             #
#####################################################################

scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


to_tensor = transforms.ToTensor()

model.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

### Get List of images:

In [13]:
list_dir = os.listdir('Database/')

In [14]:
list_dir

['009055_0M54.JPG',
 '009055_1M54.JPG',
 '019066_08M70.JPG',
 '019066_0M67.JPG',
 '020490_0M67.JPG']

In [15]:

image = Image.open('Database/'+list_dir[0])
imgblob = Variable(normalize(to_tensor(image)).unsqueeze(0))

output1 = model(imgblob)

In [16]:
np.shape(output1)

torch.Size([1, 512])

### Get Euclidean Distance of penultimal Layers between pairs of faces:

In [17]:

def RunModel(img_url_1,img_url_2):
    
    #First Image
    imagepath = img_url_1
    image = Image.open(imagepath)
    imgblob = Variable(normalize(to_tensor(image)).unsqueeze(0))
    
    output1 = model(imgblob)

    
    #Second Image
    imagepath = img_url_2
    image = Image.open(imagepath)
    imgblob = Variable(normalize(to_tensor(image)).unsqueeze(0))
    
    output2 = model(imgblob)


    dist = np.linalg.norm(output1.detach().numpy()-output2.detach().numpy())
#     print(dist)
    
    return(dist)

### Run: 

In [18]:

model_result=[]
for i in range(len(list_dir)):
    print(i)
    for j in range(len(list_dir)):
        model_result.append(RunModel('Database/'+list_dir[i],'Database/'+list_dir[j]))

0
1
2
3
4


In [11]:
for i in range(len(model_result)):
    print(model_result[i])

0.0
0.9014822
1.4664803
1.4476819
1.2412167
0.9014822
0.0
1.3835658
1.3895339
1.3140713
1.4664803
1.3835658
0.0
0.7298875
1.351997
1.4476819
1.3895339
0.7298875
0.0
1.3348677
1.2412167
1.3140713
1.351997
1.3348677
0.0


### Save:

In [67]:
np.savetxt('Euclidean_Distances_FaceNet_'+SaveAs+'.csv',np.reshape(model_result,(len(list_dir),len(list_dir))),
          delimiter=',')